# Modeling reproduction rate based on government measures in effect

In [152]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

# see all columns at once with scrolling
pd.set_option('display.max_columns', 0)

## Data
Create a table with

| Country | Date      | IndoorOver100 | MassGatherAll | StayHomeOrderPartial | ... | reproduction_rate |
|---------|-----------|---------------|---------------|----------------------|-----|-------------------|
| Austria | 3-10-2020 | 1             | 0             | 0                    |     | 1.5               |
| UK      | 1-2-2020  | 0             | 0             | 1                    |     | 0.8               |
| UK      | 2-4-2020  | 1             | 1             | 1                    |     | 0.2               |

In [177]:
owid = pd.read_csv('./data/1_owid/owid-covid-data.csv')
eu_response = pd.read_csv('./data/2_ecdc/country_response_measures.csv')

# df is the final dataset to use
df = owid[["location","date","reproduction_rate"]]
df.head()

,location,date,reproduction_rate
0,Afghanistan,2020-02-24,NaN
1,Afghanistan,2020-02-25,NaN
2,Afghanistan,2020-02-26,NaN
3,Afghanistan,2020-02-27,NaN
4,Afghanistan,2020-02-28,NaN


In [178]:
# only keep rows in df that match to a country in eu_response
df = df[[x in eu_response["Country"].unique() for x in df["location"]]]

# initialize a column for each response type with all 0s
for response in eu_response["Response_measure"].unique():
    try:
        df.insert(len(df.columns), response, 0)
    except ValueError:  # ValueError thrown if df already has that column
        pass

In [194]:
for _, row in eu_response.iterrows():
    print(row["Country"])
    print(pd.to_datetime(row["date_start"]))
    print(pd.to_datetime(row["date_end"]))
    print(row["Response_measure"])
    print(df["location"] == row["Country"]
        and pd.to_datetime(df["date"]) >= pd.to_datetime(row["date_start"])
        and pd.to_datetime(df["date"]) < pd.to_datetime(row["date_end"]))
    df[df["location"] == row["Country"]
        and pd.to_datetime(df["date"]) >= pd.to_datetime(row["date_start"])
        and pd.to_datetime(df["date"]) < pd.to_datetime(row["date_end"])][row["Response_measure"]] = 1
df

Austria
2020-03-10 00:00:00
NaT
AdaptationOfWorkplace


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [195]:
pd.to_datetime(df["date"]) >= dt.datetime(2020,8,1)

2824     False
2825     False
2826     False
2827     False
2828     False
         ...  
55012     True
55013     True
55014     True
55015     True
55016     True
Name: date, Length: 10323, dtype: bool